In [1]:
# importing all the libraries needed 
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
sns.set_style("whitegrid")
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error, mean_absolute_error

#For date time functions
from datetime import datetime
from datetime import timedelta
import math

# Importing the most popular regression libraries.
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression, ridge_regression, Lasso, SGDRegressor, Ridge
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor,StackingRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [2]:
!pip3 install lightgbm

In [3]:
train_data=pd.read_csv('train.csv',parse_dates=True)
features_data=pd.read_csv('features.csv',parse_dates=True)
stores_data=pd.read_csv('stores.csv')
test_data=pd.read_csv('test.csv')
sample_submission=pd.read_csv('sampleSubmission.csv')

In [4]:
from fbprophet import Prophet
import tqdm.notebook as tq
df = pd.DataFrame()
for i in tq.tqdm(range(1,46)):
    model=Prophet()
    filled=features_data[((features_data['Store']==i) & (features_data['Date']<'2013-05-03'))][['Date','CPI']]
    tserie = filled.rename(columns = {'Date': 'ds', 'CPI': 'y'}, inplace = False)
    tserie =tserie.sort_values(by=['ds'])
    tserie['ds'] = pd.to_datetime(tserie['ds'])
    model.fit(tserie)
    future_dates=model.make_future_dataframe(periods=13,freq = 'W',include_history =True)
    future_dates['ds'] = future_dates['ds'].apply(lambda x: x + timedelta(days=5))
    prediction=model.predict(future_dates)
    df=df.append(prediction)

df.reset_index(drop=True)
features_data['CPI1']=np.nan
for i,j in enumerate(df['yhat']):
    features_data['CPI1'].iloc[i]=j

features_data=features_data.drop(['CPI'],axis=1)
features_data = features_data.rename(columns = {'CPI1': 'CPI'})

  0%|          | 0/45 [00:00<?, ?it/s]

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disab

In [5]:
from fbprophet import Prophet
import tqdm.notebook as tq
df = pd.DataFrame()
for i in tq.tqdm(range(1,46)):
    model=Prophet()
    filled=features_data[((features_data['Store']==i) & (features_data['Date']<'2013-05-03'))][['Date','Unemployment']]
    tserie = filled.rename(columns = {'Date': 'ds', 'Unemployment': 'y'}, inplace = False)
    tserie =tserie.sort_values(by=['ds'])
    tserie['ds'] = pd.to_datetime(tserie['ds'])
    model.fit(tserie)
    future_dates=model.make_future_dataframe(periods=13,freq = 'W',include_history =True)
    future_dates['ds'] = future_dates['ds'].apply(lambda x: x + timedelta(days=5))
    prediction=model.predict(future_dates)
    df=df.append(prediction)

df.reset_index(drop=True)
features_data['Unemployment1']=np.nan
for i,j in enumerate(df['yhat']):
    features_data['Unemployment1'].iloc[i]=j

features_data=features_data.drop(['Unemployment'],axis=1)
features_data = features_data.rename(columns = {'Unemployment1': 'Unemployment'})

  0%|          | 0/45 [00:00<?, ?it/s]

INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seaso

In [6]:
stores = stores_data.merge(features_data, on ='Store' , how = 'left')
final_data_train = train_data.merge(stores, on = ['Store', 'Date', 'IsHoliday'], how = 'left')

In [7]:
stores = stores_data.merge(features_data, on ='Store' , how = 'left')
final_data_test = test_data.merge(stores, on = ['Store', 'Date', 'IsHoliday'], how = 'left')

In [8]:
print(final_data_train.shape)
print(final_data_test.shape)

(421570, 16)
(115064, 15)


In [9]:
def markdown_imputation(final_data):    
    final_data.loc[final_data.MarkDown1.isnull() ,'MarkDown1']= 0
    final_data.loc[final_data.MarkDown2.isnull() ,'MarkDown2']= 0
    final_data.loc[final_data.MarkDown3.isnull() ,'MarkDown3']= 0
    final_data.loc[final_data.MarkDown4.isnull() ,'MarkDown4']= 0
    final_data.loc[final_data.MarkDown5.isnull() ,'MarkDown5']= 0
    return final_data

In [10]:
def weekly_sales_imputation(final_data):    
    final_data[final_data.Weekly_Sales<0]
    print('before removing negative weekly sales ,shape of final dataframe is :',final_data.shape)
    final_data=final_data[final_data.Weekly_Sales>=0]
    print('after removing negative weekly sales ,shape of final dataframe is :',final_data.shape)
    return final_data

In [11]:
def train_temp_bins(final_data):
    temp_100_110_f=final_data[((final_data.Temperature>100) & (final_data.Temperature< 110))].Weekly_Sales.sum()
    temp_90_100_f=final_data[((final_data.Temperature>90) & (final_data.Temperature< 100))].Weekly_Sales.sum()
    temp_80_90_f=final_data[((final_data.Temperature>80) & (final_data.Temperature< 90))].Weekly_Sales.sum()
    temp_70_80_f=final_data[((final_data.Temperature>70) & (final_data.Temperature< 80))].Weekly_Sales.sum()
    temp_60_70_f=final_data[((final_data.Temperature>60) & (final_data.Temperature< 70))].Weekly_Sales.sum()
    temp_50_60_f=final_data[((final_data.Temperature>50) & (final_data.Temperature< 60))].Weekly_Sales.sum()
    temp_40_50_f=final_data[((final_data.Temperature>40) & (final_data.Temperature< 50))].Weekly_Sales.sum()
    temp_30_40_f=final_data[((final_data.Temperature>30) & (final_data.Temperature< 40))].Weekly_Sales.sum() 
    temp_0_30_f=final_data[((final_data.Temperature>0) & (final_data.Temperature< 30))].Weekly_Sales.sum()   
    temp_less_than_0_f=final_data[((final_data.Temperature>-10) & (final_data.Temperature< 0))].Weekly_Sales.sum()
    final_data['Temp_bins'] = np.nan
    final_data.loc[((final_data.Temperature>-10) & (final_data.Temperature<0)) ,'Temp_bins']= temp_less_than_0_f
    final_data.loc[((final_data.Temperature>0) & (final_data.Temperature< 30)) ,'Temp_bins']= temp_0_30_f
    final_data.loc[((final_data.Temperature>30) & (final_data.Temperature< 40)) ,'Temp_bins']= temp_30_40_f
    final_data.loc[((final_data.Temperature>40) & (final_data.Temperature< 50)) ,'Temp_bins']= temp_40_50_f
    final_data.loc[((final_data.Temperature>50) & (final_data.Temperature< 60)) ,'Temp_bins']= temp_50_60_f
    final_data.loc[((final_data.Temperature>60) & (final_data.Temperature< 70)) ,'Temp_bins']= temp_60_70_f
    final_data.loc[((final_data.Temperature>70) & (final_data.Temperature< 80)) ,'Temp_bins']= temp_70_80_f
    final_data.loc[((final_data.Temperature>80) & (final_data.Temperature< 90)) ,'Temp_bins']= temp_80_90_f
    final_data.loc[((final_data.Temperature>90) & (final_data.Temperature< 100)),'Temp_bins']= temp_90_100_f
    final_data.loc[((final_data.Temperature>100) & (final_data.Temperature< 110)),'Temp_bins']= temp_100_110_f
    final_data.loc[final_data.Temp_bins.isnull() ,'Temp_bins']= 0
    list1=[temp_less_than_0_f,temp_0_30_f,temp_30_40_f,temp_40_50_f,temp_50_60_f,temp_60_70_f,temp_70_80_f,temp_80_90_f,temp_90_100_f,temp_100_110_f]
    return final_data,list1

In [12]:
def test_temp_bins(final_data,list1):
    final_data['Temp_bins'] = np.nan
    final_data.loc[((final_data.Temperature>-10) & (final_data.Temperature<0)) ,'Temp_bins']= list1[0]
    final_data.loc[((final_data.Temperature>0) & (final_data.Temperature< 30)) ,'Temp_bins']= list1[1]
    final_data.loc[((final_data.Temperature>30) & (final_data.Temperature< 40)) ,'Temp_bins']= list1[2]
    final_data.loc[((final_data.Temperature>40) & (final_data.Temperature< 50)) ,'Temp_bins']= list1[3]
    final_data.loc[((final_data.Temperature>50) & (final_data.Temperature< 60)) ,'Temp_bins']= list1[4]
    final_data.loc[((final_data.Temperature>60) & (final_data.Temperature< 70)) ,'Temp_bins']= list1[5]
    final_data.loc[((final_data.Temperature>70) & (final_data.Temperature< 80)) ,'Temp_bins']= list1[6]
    final_data.loc[((final_data.Temperature>80) & (final_data.Temperature< 90)) ,'Temp_bins']= list1[7]
    final_data.loc[((final_data.Temperature>90) & (final_data.Temperature< 100)),'Temp_bins']= list1[8]
    final_data.loc[((final_data.Temperature>100) & (final_data.Temperature< 110)),'Temp_bins']= list1[9]
    final_data.loc[final_data.Temp_bins.isnull() ,'Temp_bins']= 0
    return final_data

In [13]:
def split(final_data):
    final_data['Date'] = pd.to_datetime(final_data['Date'])
    final_data['Year'] = final_data['Date'].dt.year
    final_data['Month']= final_data['Date'].dt.month
    final_data['Week'] = final_data['Date'].dt.week
    final_data['Day']  = final_data['Date'].dt.day
    return final_data

In [14]:
import datetime
def days_from_christmas_for_train(x):
    if x['Year']== 2010 :
        diff=datetime.datetime(2010, 12, 31)-x['Date']
        return diff.days
    if ((x['Year']== 2011) and (x['Date']< datetime.datetime(2011, 12, 30))):
        diff=datetime.datetime(2011, 12, 30)-x['Date']
        return diff.days
    else:
        return 0

In [15]:
import datetime
def days_from_christmas_for_test(x):
    if x['Year']== 2010 :
        diff=datetime.datetime(2010, 12, 31)-x['Date']
        return diff.days
    if ((x['Year']== 2011) and (x['Date']< datetime.datetime(2011, 12, 30))):
        diff=datetime.datetime(2011, 12, 30)-x['Date']
        return diff.days
    if ((x['Year']== 2012) and (x['Date']< datetime.datetime(2012, 12, 28))):
        diff=datetime.datetime(2012, 12, 28)-x['Date']
        return diff.days
    if ((x['Year']== 2013) and (x['Date']< datetime.datetime(2013, 12, 27))):
        diff=datetime.datetime(2013, 12, 27)-x['Date']
        return diff.days
    else:
        return 0   

In [16]:
def days_from_thanksgiving_for_train(x):
    if ((x['Year']== 2010) and (x['Date']< datetime.datetime(2010, 11, 26))):
        diff=datetime.datetime(2010, 11, 26)-x['Date']
        return diff.days
    if ((x['Year']== 2011) and (x['Date']< datetime.datetime(2011, 11, 25))):
        diff=datetime.datetime(2011, 11, 25)-x['Date']
        return diff.days
    else:
        return 0

In [17]:
def days_from_thanksgiving_for_test(x):
    if ((x['Year']== 2010) and (x['Date']< datetime.datetime(2010, 11, 26))):
        diff=datetime.datetime(2010, 11, 26)-x['Date']
        return diff.days
    if ((x['Year']== 2011) and (x['Date']< datetime.datetime(2011, 11, 25))):
        diff=datetime.datetime(2011, 11, 25)-x['Date']
        return diff.days
    if ((x['Year']== 2012) and (x['Date']< datetime.datetime(2012, 11, 23))):
        diff=datetime.datetime(2012, 11, 23)-x['Date']
        return diff.days
    if ((x['Year']== 2013) and (x['Date']< datetime.datetime(2013, 11, 29))):
        diff=datetime.datetime(2013, 11, 29)-x['Date']
        return diff.days
    else:
        return 0

In [18]:
def holiday_type(x):
    if   (x['IsHoliday']== 1) & (x['Week']==6):
        return 1 #SuperBowl
    elif (x['IsHoliday']== 1) & (x['Week']==36):
        return 2 #LaborDay
    elif (x['IsHoliday']== 1) & (x['Week']==47):
        return 3 #Thanksgiving
    elif (x['IsHoliday']== 1) & (x['Week']==52):
        return 4 #Christmas
    else:
        return 0

In [19]:
def holiday_label(final_data):
    final_data.loc[(final_data.IsHoliday==True) ,'IsHoliday']= 1
    final_data.loc[(final_data.IsHoliday==False) ,'IsHoliday']= 0
    return final_data

In [20]:
def type_label(final_data):
    final_data.loc[(final_data.Type=='A') ,'Type']= 1
    final_data.loc[(final_data.Type=='B') ,'Type']= 2
    final_data.loc[(final_data.Type=='C') ,'Type']= 3
    return final_data

In [21]:
import holidays
def holiday_in_week_train(final_data):
    dates =[]
    for ptr in holidays.US(years = 2010).items():
        dates.append(ptr[0])
    for ptr in holidays.US(years = 2011).items():
        dates.append(ptr[0])
    for ptr in holidays.US(years = 2012).items():
        dates.append(ptr[0])
    holiday_count=[] 
    for index, row in final_data.iterrows():
        dat = final_data['Date'][index]
        dt=[]
        for i in range(0,5):
            dt.append(dat - datetime.timedelta(days = i))
        for i in range(1,3):
            dt.append(dat + datetime.timedelta(days = i))
        count = 0
        for date in dates:
            if date in dt:
                count +=1
        holiday_count.append(count)
    return holiday_count  

In [22]:
import holidays
def holiday_in_week_test(final_data):
    dates =[]
    for ptr in holidays.US(years = 2010).items():
        dates.append(ptr[0])
    for ptr in holidays.US(years = 2011).items():
        dates.append(ptr[0])
    for ptr in holidays.US(years = 2012).items():
        dates.append(ptr[0])
    for ptr in holidays.US(years = 2013).items():
        dates.append(ptr[0])
    holiday_count=[] 
    for index, row in final_data.iterrows():
        dat = final_data['Date'][index]
        dt=[]
        for i in range(0,5):
            dt.append(dat - datetime.timedelta(days = i))
        for i in range(1,3):
            dt.append(dat + datetime.timedelta(days = i))
        count = 0
        for date in dates:
            if date in dt:
                count +=1
        holiday_count.append(count)
    return holiday_count  

## Training data preprocessing and generating new features

In [23]:
final_data_train=markdown_imputation(final_data_train)
final_data_train=weekly_sales_imputation(final_data_train)
final_data_train,list1=train_temp_bins(final_data_train)
final_data_train=split(final_data_train)
final_data_train['diff_from_christmas'] = final_data_train.apply(days_from_christmas_for_train, axis=1)  
final_data_train['days_from_thanksgiving'] = final_data_train.apply(days_from_thanksgiving_for_train, axis=1)  
final_data_train['IsHoliday_bins'] = final_data_train.apply(holiday_type, axis=1)
final_data_train=holiday_label(final_data_train)
final_data_train=type_label(final_data_train)
final_data_train['Holidays'] = np.array(holiday_in_week_train(final_data_train))

before removing negative weekly sales ,shape of final dataframe is : (421570, 16)
after removing negative weekly sales ,shape of final dataframe is : (420285, 16)


## Test data preprocessing and generating new features

In [24]:
final_data_test=markdown_imputation(final_data_test)
final_data_test=test_temp_bins(final_data_test,list1)
final_data_test=split(final_data_test)
final_data_test['diff_from_christmas'] = final_data_test.apply(days_from_christmas_for_test, axis=1) 
final_data_test['days_from_thanksgiving'] = final_data_test.apply(days_from_thanksgiving_for_test, axis=1)   
final_data_test['IsHoliday_bins'] = final_data_test.apply(holiday_type, axis=1)
final_data_test=holiday_label(final_data_test)
final_data_test=type_label(final_data_test)
final_data_test['Holidays'] = np.array(holiday_in_week_test(final_data_test))

In [25]:
final_data_train=final_data_train.reset_index(drop=True)
final_data_test=final_data_test.reset_index(drop=True)

In [26]:
def WMAE(dataset, real, predicted):
  
    weights = dataset.IsHoliday.apply(lambda x: 5 if x else 1)
    return np.round(np.sum(weights*abs(real-predicted))/(np.sum(weights)), 2)

In [27]:
# importing datetime module
from datetime import *

In [31]:
from fbprophet import Prophet
import datetime
regressors = ['Store','Dept','IsHoliday','Size','Week','Type','Year','Holidays','Day']
def Facebook_FbProphet(df1,number_of_pred,new):
    shape=df1.shape[0]
    df2=df1
    model4=Prophet()
    
    for regressor in regressors:
        model4.add_regressor(name = regressor, standardize = False)
    tserie = df2.rename(columns = {'Date': 'ds', 'Weekly_Sales': 'y'}, inplace = False)
    tserie['ds'] = pd.to_datetime(tserie['ds'])
    model4.fit(tserie)
    future_dates=model4.make_future_dataframe(periods=208,freq = 'W',include_history =True) 
    def add(x):
        if x.strftime('%Y-%m-%d') > '2012-10-26' :
            a=x + timedelta(days=5)
            return a.strftime('%Y-%m-%d')
        else:
            return x
    future_dates['ds'] = future_dates['ds'].apply(lambda x: add(x))
    new=new[['Date','Store','Dept','IsHoliday','Size','Week','Type','Year','Holidays','Day']] 
    new = new.rename(columns = {'Date': 'ds'}, inplace = False)
    new['ds'] = pd.to_datetime(new['ds'])
    tserie=tserie[['ds','Store','Dept','IsHoliday','Size','Week','Type','Year','Holidays','Day']] 
    tserie=tserie.append(new)
    future_dates = future_dates.merge(tserie, how='inner', on = ['ds'])
    predicted = model4.predict(future_dates)
    yhat = predicted['yhat']
    return list(yhat[shape:])

In [32]:
from tqdm import tqdm
store_num=[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
        35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45]
dept_num=[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 16, 17, 18,
        19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52,
        54, 55, 56, 58, 59, 60, 65, 67, 71, 72, 74, 77, 78, 79, 80, 81, 82,
        83, 85, 87, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
predicted_y=[]
p=0
for i in tqdm(store_num):
    for j in dept_num:
        print(i,j)
        df=final_data_train[(final_data_train['Store']==i) & (final_data_train['Dept']==j)]
        number_of_pred=test_data[(test_data['Store']==i) & (test_data['Dept']==j)]['Store'].count()
        new=final_data_test[(final_data_test['Store']==i) & (final_data_test['Dept']==j)]
        extra_pred=number_of_pred-df.shape[0]
        if extra_pred <= 0:
            if df.shape[0]>2:
                predicted_y=predicted_y+Facebook_FbProphet(df,number_of_pred,new)
            else:
                list1=[]
                for k in range(0,number_of_pred):
                    list1.append(0)
                predicted_y=predicted_y+list1
        else:    
            list2=[]
            for k in range(0,number_of_pred):
                list2.append(0)
            predicted_y=predicted_y+list2  

  0%|          | 0/45 [00:00<?, ?it/s]INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


1 1


INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


1 2


INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

45 96
45 97


INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


45 98


100%|██████████| 45/45 [2:58:07<00:00, 237.50s/it]

45 99


In [89]:
len(predicted_y)

115064

In [114]:
new=predicted_y

In [122]:
y_hat=[]
import numpy as np
for m,n in enumerate(predicted_y):
    if n==0:
        store_num=test_data.iloc[m]['Store']
        dept_num=test_data.iloc[m]['Dept']
        actual=final_data_train[(final_data_train['Store']==store_num) & (final_data_train['Dept']==dept_num)]['Weekly_Sales'].mean()
        if actual>0:
            y_hat.append(actual)
        else:
            c=0
            a=0
            for i in range(1,46):
                b=final_data_train[(final_data_train['Store']==i) & (final_data_train['Dept']==dept_num)]['Weekly_Sales'].mean()
                if b>0:
                    c=c+1    
                    a=a+b
            y_hat.append(a/c)
    else:
        y_hat.append(n)

In [123]:
sample_submission=pd.read_csv('sampleSubmission.csv')
sample_submission['Weekly_Sales']=y_hat
sample_submission.to_csv('submission.csv',index=False)